In [1]:
from pathlib import Path
from splitpy import arguments, utils
from splitpy import Split, DiagPlot, PygmtPlot
import matplotlib.pyplot as plt
import numpy as np
import pickle
import stdb
from obspy.clients.fdsn import Client
from obspy import UTCDateTime, read
import matplotlib
import os, glob
matplotlib.use('Qt5Agg')
import warnings
warnings.filterwarnings('ignore')

In [2]:
datapath = 'Model_mseed'
# timelist = ['20220524_135252','20220524_135300','20220524_135310']

In [3]:
class Meta(object):
    def __init__(self,time,baz=0):
        self.time = UTCDateTime(time)
        self.baz = int(baz)
        self.lon = 0
        self.lat = 0
        self.dep = 100
        self.mag = 7.0
        self.epi_dist = 120*111
        self.gac = 120
        self.az = 360-int(baz)
        self.ttime = ttime
        self.slow = 0.04
        self.inc = 0
        self.phase = 'SKS'
        self.accept = True
        self.snrq = 10.0
        self.snrt = 5.0
        self.maxdt = 4.0
        self.ddt = 0.1
        self.dphi = 1
        self.align = 'LQT'
        self.rotated = False
        self.CpH = None
        self.fmin=0.05
        self.fmax=0.125

In [9]:

db = stdb.io.load_db(fname='XX_MOD.pkl')
stkey = 'XX.MOD'
sta = db[stkey]
split= Split(sta)
for timedir  in glob.glob(f'{datapath}/*104127'):
    print(timedir)
    time = timedir.rsplit('/')[-1]
    ttime = 47 ###SKS
    tmin=ttime-10
    tmax=ttime+20
    datadir = f'{datapath}/{time}'
    baz = 270


    split.meta = Meta(time,baz)
    
    stafile = f'{datadir}/Station_data.pkl'
    ZNEfile = f'{datadir}/ZNE_data.pkl'
    LQTfile = f'{datadir}/LQT_data.pkl'
    metafile = f'{datadir}/Meta_data.pkl'
    splitfile = f'{datadir}/Split_results_auto.pkl'
    RCfile = f'{datadir}/RC_res.pkl'
    SCfile = f'{datadir}/SC_res.pkl'
    pickle.dump(split.sta, open(stafile, "wb"))

    st = read(f'{datadir}/*.mseed')
    split.dataZNE = st
    pickle.dump(split.dataZNE, open(ZNEfile, "wb"))
    split.dataLQT = st.copy()
    
    split.dataLQT.rotate(method='ZNE->LQT', back_azimuth=split.meta.baz, inclination=split.meta.inc)
    split.dataLQT.filter('bandpass',freqmin=split.meta.fmin, freqmax=split.meta.fmax, corners=4, zerophase=True)
    pickle.dump(split.dataLQT, open(LQTfile, "wb"))     
    
    split.analyze(t1=split.meta.time+tmin, t2=split.meta.time+tmax, verbose=False, taper=True)
    split.is_null(3)
    split.get_quality(verbose=False)
    split.particle_PCA(freqmin=split.meta.fmin, freqmax=split.meta.fmax, t1=split.meta.time+tmin, t2=split.meta.time+tmax)
    pickle.dump(split.RC_res, open(RCfile, "wb"))
    pickle.dump(split.SC_res,open(SCfile, "wb"))
    pickle.dump(split.meta, open(metafile, "wb"))
    dplotfile = f'{datadir}/Plot_diagnostic_auto.png'
    dplot = DiagPlot(split)
    dplot.plot_diagnostic(calc_type='auto', t1=split.meta.time+tmin, t2=split.meta.time+tmax)
    plt.figure(dplot.axes[0].number)
    dplot.save(dplotfile)
    dplotfile = f'{datadir}/Plot_diagnostic_auto.pdf'
    dplot.save(dplotfile)
    print('======================finish')
    # except:
        # print(f'{time}° error')
# else: print('done already! ')

Model_mseed/20221114_104127
======================finish
